## FiLM layer

In [1]:
import numpy as np
import tensorflow.keras
from tensorflow import keras
import torch
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Conv2D, MaxPooling2D,ReLU
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
import IPython.display as ipd
from kymatio import Scattering1D
import hitdifferentparts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pescador
import random
import os
import librosa
import pickle
import matplotlib.pyplot as plt
import math

## load and preprocess data

In [148]:
#download an example file from hpc
pkl_trainpath = "./scattering_fold-train_J-08_Q-01_order2.pkl"
pkl_valpath = "./scattering_fold-val_J-08_Q-01_order2.pkl"
#pkl_testpath = "./scattering_fold-test_J-08_Q-02_order2.pkl"
pkl_train = open(pkl_trainpath, 'rb')
#pkl_test = open(pkl_testpath, 'rb')
pkl_val = open(pkl_valpath, 'rb')

Sy_train,y_train = pickle.load(pkl_train) 
#Sy_test,y_test = pickle.load(pkl_test) 
Sy_val,y_val = pickle.load(pkl_val) 

In [149]:
eps =1e-11
Sy_train_log2 = np.log1p(((Sy_train>0)*Sy_train)/eps)
Sy_val_log2 = np.log1p(((Sy_val>0)*Sy_val)/eps)
#Sy_test_log2 = np.log1p((Sy_test>0)*Sy_test/eps)

#log scale p and D
for idx in range(2,4):
    y_train[:,idx] = [math.log10(i) for i in y_train[:,idx]]
    #y_test[:,idx] = [math.log10(i) for i in y_test[:,idx]]
    y_val[:,idx] = [math.log10(i) for i in y_val[:,idx]]

# normalization of the physical parameters
scaler = MinMaxScaler()
scaler.fit(y_train[:-2])
y_train_normalized = scaler.transform(y_train[:-2])
y_val_normalized = scaler.transform(y_val[:-2])
#y_test_normalized = scaler.transform(y_test[:-2])

In [246]:
print(Sy_train.shape,y_train.shape)

(82224, 128, 43) (82224, 7)


## make a customized FiLM
Mx(t,p) = (1+f(u,p)) Sx(t,p) + g(u,p) 
1. x(t) is the drum sound corresponding to a stroke location u and shape theta
2. Sx(t,p) is the scattering transform of x at the scattering path p=(j1, j2)
3. f and g are functions learned by FiLM. They take the location u as an input and return a different number for each scattering path p. The effect of f is multiplicative while the effect of g is additive
4. Mx(t,p) is result of feature-wise linear modulation (FiLM). It has the same dimensionality as the scattering transform Sx, but is conditioned on stroke location u

In [171]:
input_shape = [(None,128,43),(None,2)]
Sc_input_shape, u_input_shape = input_shape
print(Sc_input_shape,u_input_shape)
height, width = Sc_input_shape[1:] # t, p , 2
FiLM_tns = u_input_shape[1]
n_feature_maps = width
print((FiLM_tns, int(2*n_feature_maps)))

(None, 128, 43) (None, 2)
(2, 86)


In [266]:
#kernel (bs,i,2p), u (bs,i), p=2,bs=5, want the output to shape as (bs,2p)
F = np.arange(40).reshape(5,2,4)
u = np.arange(10).reshape(5,2,1)
tf.keras.layers.Dot(axes=(1))([u, F])
#(5,1,2),(5,1,2) need to time (bs,t,p) = (5,t,2)

<tf.Tensor: id=8095, shape=(5, 1, 4), dtype=int64, numpy=
array([[[  4,   5,   6,   7]],

       [[ 52,  57,  62,  67]],

       [[164, 173, 182, 191]],

       [[340, 353, 366, 379]],

       [[580, 597, 614, 631]]])>

In [291]:
#no activation included FiLM layer - has to add it manually afterwards
class FiLM(keras.layers.Layer):

    def __init__(self, **kwargs):
        super(FiLM, self).__init__(**kwargs)

    def build(self, input_shape): # input shape = (None,t,p,2)
       
        Sc_input_shape, u_input_shape = input_shape
       # print(Sc_input_shape)
        self.height, self.width = Sc_input_shape[1:] # t, p , 2
        FiLM_tns = u_input_shape[1]
       # print(FiLM_tns,u_input_shape)
        self.n_feature_maps = self.width
        
        #initialize trainable weights, should be 2*2p? f_0 + f_1 u_1 _ f_2 u_2
        self.kernel = self.add_weight(name = 'kernel', 
                                      shape = (FiLM_tns, int(2*self.n_feature_maps)),
                                      initializer = 'normal', trainable = True) 
        
        #assert(int(2 * self.n_feature_maps)==FiLM_tns_shape[1]) #film tensor size need to be len(u) by 2p 
        super(FiLM, self).build(input_shape)

    def call(self, x):
        #assert isinstance(x, list)
        conv_output, FiLM_tns = x # x = [Sx,u]; [t by p, length-2]
        #FiLM.append(0) # to include the bias term
        #(bs,2) dot (bs,2,2p) = (bs,2p)
        FiLM_tns = K.dot(FiLM_tns,self.kernel) # u -> [f(u),g(u)] now it's (bs,1,2p)
        print("before",FiLM_tns.shape)
        #put [f(u),g(u)] in the fourth dimension
        FiLM_tns = K.expand_dims(FiLM_tns, axis=[1]) 
        #FiLM_tns = K.expand_dims(FiLM_tns, axis=[1]) #make it into [1, 1, 1,2p]
        FiLM_tns = K.tile(FiLM_tns, [1, self.height, 1]) #[1,Sx.shape[0],Sx.shape[1],2p]
        print(FiLM_tns.shape)
        #extract f(u) and g(u)
        gammas = FiLM_tns[:, :, :self.n_feature_maps] 
        betas = FiLM_tns[:, :, self.n_feature_maps:]
        
        # Apply affine transformation
        return (1 + gammas) * conv_output + betas

    def compute_output_shape(self, input_shape):
        #assert isinstance(input_shape, list)
        #return (input_shape[1],input_shape[2],self.n_feature_maps) # 
        return input_shape[:-1]
    
    
    
    

In [279]:
Sc_input_shape = (Sy_train.shape[1],Sy_train.shape[2]) 
u_input_shape = (2,)

Sc_input = keras.layers.Input(shape=Sc_input_shape)
u_input = keras.layers.Input(shape=u_input_shape)

#input_model = keras.layers.Concatenate()([Sc_input, u_input])
x = FiLM(input_shape = [Sc_input_shape,u_input_shape],name='FiLM_layer',dynamic=True)([Sc_input,u_input])
print(x[0].shape)
#x = Dense(20)(x)
#model.add(BatchNormalization(input_shape=Sc_input_shape))
x = BatchNormalization()(x[0])
x = Conv1D(filters=16,
        kernel_size=(8,), padding="same",name='conv1')(x)
model = Model(inputs=[Sc_input, u_input], outputs=x)
model.summary()

(None, 128, 43)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 43)]    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
FiLM_layer (FiLM)               [(None, 128, 43)]    172         input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 128, 43)      172         FiLM_layer[

In [292]:
def create_model_adjustable(J,Q,order,k_size,nchan_out,activation):
    N = 2**15
    y = np.random.rand(N)
    scattering = Scattering1D(J = J,shape=(N,), Q = Q, max_order=order) 
    Sy = np.array(scattering(torch.Tensor(y))).T
    nchan_in = 1       # number of input channels.  1 since it is BW

    #initialize input sizes
    Sc_input_shape = Sy.shape
    u_input_shape = (2,)

    
    input_shape = [Sc_input_shape,u_input_shape]#Sy.shape
    kernel_size = (k_size,)
    
    
    K.clear_session()
    #define input
    Sc_input = keras.layers.Input(shape=Sc_input_shape)
    u_input = keras.layers.Input(shape=u_input_shape)
    
    x = FiLM(input_shape = [Sc_input_shape,u_input_shape],name='FiLM_layer',dynamic=True)([Sc_input,u_input])
    
    #1 conv layer +  1 batch normalization + nonlinear activation + pooling
    x = BatchNormalization()(x[0])
    
    x = Conv1D(filters=nchan_out,
        kernel_size=kernel_size, padding="same",name='conv1')(x)
    x = Activation("relu")(x)
    #model.add(BatchNormalization())
    #model.add(Activation("relu"))

    if x[0].shape[1]>=4:
        pool = 4
    elif x[0].shape[1]==2:
        pool = 2

    #model.add(AveragePooling1D(pool_size=(pool,)))
    x = AveragePooling1D(pool_size=(pool,))(x)

    for i in range(3):
        
        #model.add(Conv1D(filters=nchan_out,
        #             kernel_size=kernel_size, padding="same" ))
        
        x = Conv1D(filters=nchan_out,
                     kernel_size=kernel_size, padding="same" )(x)
        
        x = BatchNormalization()(x)
        #model.add(BatchNormalization())
        x = Activation("relu")(x)
        #print(x)
        #model.add(Activation("relu"))
        #print('before pool',model.layers[-1].output_shape)
        if x.shape[1] >= 4:
            x = AveragePooling1D(pool_size=(4,))(x)
            #model.add(AveragePooling1D(pool_size=(4,)))
        elif x.shape[1] == 2:
            x = AveragePooling1D(pool_size=(2,))(x)
            #model.add(AveragePooling1D(pool_size=(2,)))
        #print(model.layers[-1].output_shape)

    #model.add(BatchNormalization())
    x = BatchNormalization()(x)
    x = Flatten()(x)
    #model.add(Flatten())
    x = Dense(64, activation='relu')(x)
    #model.add(Dense(64, activation='relu'))
    x = BatchNormalization()(x)
    #model.add(BatchNormalization())
    #what activation should be chosen for last layer, for regression problem? should be a linear function
    x = Dense(5, activation=activation)(x)
   # model.add(Dense(5, activation=activation)) #output layer that corresponds to the 5 physical parameters.


    # Compile the model
    model = Model(inputs=[Sc_input, u_input], outputs=x)
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])




    return model

In [293]:
model2 = create_model_adjustable(8,1,2,k_size=8,nchan_out=16,activation="linear")
#model2.summary()

In [297]:
y_train_theta.shape

(82224, 7)

In [298]:
Sy_train = np.asarray(Sy_train).astype(np.float32)
y_train_u = np.asarray(y_train[:,-2:]).astype(np.float32)
y_train_theta = np.asarray(y_train[:,:-2]).astype(np.float32)
hist = model2.fit([Sy_train,y_train_u],
                y_train_theta,
                epochs=1,
                verbose=2,
                batch_size=64,
                #validation_data = (Sy_val_log2[:,-round(Sy_val.shape[1] * 1):,:],y_val),
                use_multiprocessing=False)

validation_loss = hist.history['val_loss'][0]

Train on 82224 samples
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)
before (64, 86)
(64, 128, 86)


KeyboardInterrupt: 

In [ ]:

trial_dir = "../output/10trials_"+str(index)+"/tests"+str(trial)+"/"

os.makedirs(trial_dir, exist_ok=True)
best_validation_loss = np.inf
zoom_factor = 1
n = Sy_train.shape[0]
shape_time = round(Sy_train.shape[1] * zoom_factor)
steps_per_epoch = 50
bs = 64
m = bs*steps_per_epoch
idx = np.arange(0,n,1)
val_loss=[]
train_loss = []
test_loss = []
model_adjustable = create_model_adjustable(J=J,Q=Q,order=order,k_size=8,nchan_out=16,activation='linear')
save_log = os.path.join(trial_dir,pickle_name+"_score.pkl")
#model_adjustable.summary()
print('Start fitting the model...')
for epoch in range(30):
	np.random.shuffle(idx)
	Sy_temp = Sy_train_log2[idx[:m],:shape_time,:]
	y_temp = y_train_normalized[idx[:m],:]

	hist = model_adjustable.fit(Sy_temp,
				y_temp,
				epochs=1,
				verbose=2,
				batch_size=bs,
				validation_data = (Sy_val_log2[:,-shape_time:,:],y_val_normalized),
				use_multiprocessing=False)

	validation_loss = hist.history['val_loss'][0]

	test_loss.append(model_adjustable.evaluate(Sy_test_log2,y_test_normalized)[0])
	val_loss.append(validation_loss)
	train_loss.append(hist.history['loss'][0])

	if validation_loss < best_validation_loss:
		best_validation_loss = validation_loss
		#epoch_str = "epoch-" + str(epoch).zfill(3)
		epoch_network_path = os.path.join(
		   trial_dir, "_".join([ "J-" + str(J).zfill(2), "Q-" + str(Q).zfill(2), "order" + str(order)]) + ".h5")
		model_adjustable.save(epoch_network_path)



		with open(save_log, 'wb') as filehandle:
		    # store the data as binary data stream
		    pickle.dump([val_loss[-1],train_loss[-1],test_loss[-1]], filehandle)

print('Finished!')


In [112]:
N = 2 ** 15
J=6
order=1
Q = 1
scattering = Scattering1D(J=J, shape=(N,), Q=Q, max_order=order)



waveform,_ = librosa.load("./3643_sound.wav")
torch_waveform = torch.Tensor(waveform)
Sx = np.array(scattering(torch_waveform).T)

In [113]:
df = pd.read_csv("./diffshapes_param.csv")
sample_ids = df.values[:, 0]

In [114]:
y = df.values[:, 1:-1]